In [1]:
import pandas as pd

# Read the lookup table
data = pd.ExcelFile('lookupTable.xlsx')
lookupTable = pd.read_excel(data, 'previousVMMCSubmissions')
typesLookup = pd.read_excel(data,'types')

# Read the mcs dataset
dashboard = pd.ExcelFile("data.xlsx")

mcs = pd.read_excel(dashboard,"Statistics")

# Clean column names in lookupTable (remove 'field: ' part)
lookupTable.columns = [col.split(":")[1].strip() if ':' in col else col for col in lookupTable.columns]


# Identify common columns between lookupTable and mcs
common_columns = [col for col in mcs.columns if col in lookupTable.columns]

# Subset mcs to keep only the common columns
mcs_cleaned = mcs[common_columns]

# If lookupTable is empty, append rows from mcs_cleaned
if lookupTable.empty:
    lookupTable = mcs_cleaned.copy()
else:
    # If lookupTable is not empty, append the rows in mcs_cleaned that don't already exist
    lookupTable = pd.concat([lookupTable, mcs_cleaned]).drop_duplicates(subset=['facilityName', 'recordingMonth', 'year'], keep='last')

# Drop rows with NaN values
lookupTable = lookupTable[lookupTable['facilityName'].notna()]

lookupTable.columns = [
    'field: ' + col if col not in ['Delete(Y/N)', 'UID'] else col
    for col in lookupTable.columns
]

# Fill missing values with 0 for all columns except 'Delete(Y/N)' and 'field: UID'
columns_to_exclude = ['Delete(Y/N)', 'UID']

# Fill missing values with 0 in the columns that are not in the exclusion list
lookupTable = lookupTable.apply(lambda x: x.fillna(0) if x.name not in columns_to_exclude else x)

# Assign 'N' to every cell in the 'field: Delete(Y/N)' column
lookupTable['Delete(Y/N)'] = 'N'


# Now save the DataFrame to Excel
with pd.ExcelWriter('ZDIPlookup.xlsx', engine='xlsxwriter') as writer:
    # Save the sorted general data to the first sheet
    typesLookup.to_excel(writer, sheet_name='types', index=False)
    
    # Save the sorted AE data to a new sheet
    lookupTable.to_excel(writer, sheet_name='previousVMMCSubmissions', index=False)

In [2]:
lookupTable.to_excel('resultantLookupTable.xlsx')
lookupTable

UID Delete(Y/N) field: recordingMonth  field: year  \
0    NaN           N                 April       2025.0   
1    NaN           N                   May       2025.0   
2    NaN           N                 April       2025.0   
3    NaN           N                  June       2025.0   
4    NaN           N                   May       2025.0   
..   ...         ...                   ...          ...   
527  NaN           N                  June       2025.0   
528  NaN           N                   May       2025.0   
529  NaN           N                 April       2025.0   
530  NaN           N                  June       2025.0   
531  NaN           N                   May       2025.0   

              field: facilityName  field: mc15-19  field: mc20-24  \
0                    Angwa Clinic             3.0             5.0   
1                    Angwa Clinic             1.0             3.0   
2    Arcadia Rural Council Clinic            16.0            10.0   
3    Arcadia Rural Council Clinic            15.0             5.0   
4    Arcadia Rural Council Clinic            15.0             7.0   
..                            ...             ...             ...   
527   Zibwowa Rural Health Centre            10.0             3.0   
528   Zibwowa Rural Health Centre             5.0             5.0   
529   Zvipani Rural Health Centre             1.0             7.0   
530   Zvipani Rural Health Centre             1.0             8.0   
531   Zvipani Rural Health Centre             4.0             6.0   

     field: mc25-29  field: mc30-34  field: mc35-39  ...  field: fu45-49  \
0               0.0             1.0             1.0  ...             1.0   
1               4.0             2.0             1.0  ...             0.0   
2               4.0             0.0             0.0  ...             0.0   
3               2.0             0.0             0.0  ...             0.0   
4               2.0             0.0             0.0  ...             0.0   
..              ...             ...             ...  ...             ...   
527             0.0             0.0             0.0  ...             0.0   
528             1.0             0.0             0.0  ...             0.0   
529             3.0             0.0             1.0  ...             0.0   
530             6.0             2.0             0.0  ...             0.0   
531             6.0             3.0             1.0  ...             0.0   

     field: fu50  field: total_hiv_negative_linked_to_prep  \
0            0.0                                       0.0   
1            0.0                                       0.0   
2            0.0                                       0.0   
3            0.0                                       0.0   
4            0.0                                       0.0   
..           ...                                       ...   
527          0.0                                       0.0   
528          0.0                                       0.0   
529          0.0                                       0.0   
530          0.0                                       0.0   
531          0.0                                       0.0   

     field: total_hiv_positive_linked_to_care  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   
..                                        ...   
527                                       0.0   
528                                       0.0   
529                                       0.0   
530                                       0.0   
531                                       0.0   

     field: total_mcs_referred_for_srh_services  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3               